# **YAI 2021 Fall Project - Medical Project Team**

**Participants**
* Dongha Kim - Yonsei Univ. College of Medicine.
* Donggeon Bae - Yonsei. Univ. Dept. of Electrical and Electronic Engineering.
* Junho Lee - Yonsei Univ. Dept. of Computer & Telecommunications Engineering.


## Description

TBD


## Dataset Information

**Cancer Instance Segmentation and Classification: kaggle**

19 Tissue types (Breast, Colon, Bile-duct, Esophagus, Uterus, Lung, Cervix, Head&Neck, Skin, Adrenal Gland, Kidney, Stomach, Prostate, Testis, Liver, Thyroid, Pancreas, Ovary, Bladder). Note, that it also unifies existing datasets within it, we have carefully labelled these under a single nuclei categories schema that is common to all 19 tissues.

* https://www.kaggle.com/andrewmvd/cancer-inst-segmentation-and-classification
* https://www.kaggle.com/andrewmvd/cancer-instance-segmentation-and-classification-2
* https://www.kaggle.com/andrewmvd/cancer-instance-segmentation-and-classification-3


## Model Architecture

**Baseline: DeepLabV3 + Resnet101**
* https://pytorch.org/hub/pytorch_vision_deeplabv3_resnet101/


## Full Source Code

* github link: TBD

All non-necessary codes are modularized as package. Watch all codes in github repository.

# Runtime Preparation

## Mount Data Drive

In [ ]:
!git clone https://github.com/kdha0727/cancer-instance-segmentation-from-tissue.git
import os
import sys
sys.path.insert(0, os.path.join(os.getcwd(), 'cancer-instance-segmentation-from-tissue'))
try:
    from google.colab import drive  # NOQA
except ImportError:
    raise RuntimeError("This notebook must be run on colab runtime!")
else:
    drive.mount('/content/drive')  # NOQA
    %cd "/content/drive/Shareddrives/YAI 2021 가을학기 의료 프로젝트팀"

Cloning into 'cancer-instance-segmentation-from-tissue'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 54 (delta 26), reused 48 (delta 20), pack-reused 0
Unpacking objects: 100% (54/54), done.
Mounted at /content/drive
/content/drive/Shareddrives/YAI 2021 가을학기 의료 프로젝트팀


## View Runtime Information

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0 or gpu_info.find('not found') >= 0:
    import os
    if 'TPU_NAME' in os.environ:
        mode = 'xla'; print('TPU Runtime')
    else:
        mode = 'cpu'; print('Not connected to a GPU')
else:
    mode = 'cuda'; print(gpu_info)

Mon Oct  4 18:22:36 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print(f'Your runtime has {ram_gb:.1f} gigabytes of available RAM\n'
      f'{"Not" if ram_gb < 20 else "You are"} using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM
Not using a high-RAM runtime!


In [ ]:
import sys
import platform
print(f"OS version: \t\t{platform.platform()}\nPython version:\t\t{sys.version.replace(chr(10), str())}")

OS version: 		Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic
Python version:		3.7.12 (default, Sep 10 2021, 00:21:48) [GCC 7.5.0]


## Prepare device and library

In [ ]:
# Prepare device

if mode == 'xla':
    try:
        import torch_xla
    except ImportError:
        !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
        !python pytorch-xla-env-setup.py
        !rm -rf pytorch-xla-env-setup.py *.whl
        import torch_xla
    import torch
    import torch_xla.core.xla_model as xm
    # Acquires the default Cloud TPU core and moves the model to it
    device = xm.xla_device()

elif mode == 'cuda':
    import torch
    device = torch.device("cuda")
    # loader_kwargs = dict(pin_memory=True)

else:
    import torch
    device = torch.device("cpu")

print(f"Torch version:\t\t{torch.__version__}\nTorch device:\t\t{device}")

Torch version:		1.9.0+cu102
Torch device:		cuda


In [ ]:
# After all installation, import all libraries used.

!pip install torchinfo
!pip install pyclean
!pyclean .

import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import ChainDataset, RandomSampler, DataLoader
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
import torchinfo
from tqdm.notebook import tqdm
# import albumentations as A
# from albumentations.pytorch.transforms import ToTensorV2


Cleaning directory .
Total 0 files, 0 directories removed.


# Dataset Preparation

**Dataset Information**


19 Tissue types (Breast, Colon, Bile-duct, Esophagus, Uterus, Lung, Cervix, Head&Neck, Skin, Adrenal Gland, Kidney, Stomach, Prostate, Testis, Liver, Thyroid, Pancreas, Ovary, Bladder). Note, that it also unifies existing datasets within it, we have carefully labelled these under a single nuclei categories schema that is common to all 19 tissues.

**This particular directory includes training patches of size 256x256 and their masks, this is one of the folds. In total there are more than 7 thousand training patches within all of the three folds.**

The files within each fold directories are:

* `images.npy` - image patches of 256x256

* `masks.npy` an array of 6 channel instance-wise masks (0: Neoplastic cells, 1: Inflammatory, 2: Connective/Soft tissue cells, 3: Dead Cells, 4: Epithelial, 6: Background)

* `types.py`  tissue type that a given path was extracted from.

**The `masks/` directory and it's contents are licensed under a Creative Commons [Attribution-NonCommercial-ShareAlike 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)  license**

## Make Dataset Class

In [ ]:
from datasets.numpy_lazy import LazyNumpyDataset

## Instantiate Dataset

In [ ]:
!cp --verbose -r data /content/data

^C


In [ ]:
batch_length = [886, 885, 885, 841, 841, 841, 908, 907, 907]

train_subset = [0, 2, 3, 4, 6, 8]
val_subset = [1, 5]
test_subset = [7]

path_format = os.path.join("/content/data", "processed", "{0}", "{1}.npy")
train_dataset = ChainDataset([
    LazyEvalNumpyDataset(
        image_path=path_format.format("images", i),
        mask_path=path_format.format("masks", i),
        length=batch_length[i],
        sampler_class=RandomSampler,  # issue
    ) for i in train_subset
])
val_dataset = ChainDataset([
    LazyEvalNumpyDataset(
        image_path=path_format.format("images", i),
        mask_path=path_format.format("masks", i),
        length=batch_length[i],
    ) for i in val_subset
])
test_dataset = ChainDataset([
    LazyEvalNumpyDataset(
        image_path=path_format.format("images", i),
        mask_path=path_format.format("masks", i),
        length=batch_length[i],
    ) for i in test_subset
])

# Network Preparation

## Segmentation Network

**Baseline Model**: DeepLabV3-Resnet101

Other Architectures

* A-HRNet: https://ieeexplore.ieee.org/document/9253142
* See Also: https://arxiv.org/pdf/2005.10821.pdf

In [ ]:
from models.unet import UNet, InceptionUNet, SpectralPool2d, HydPool2d
from models.refinenet import refinenet50, refinenet101, refinenet152, rf_lw50, rf_lw101, rf_lw152

In [ ]:
from torchvision.models.segmentation.deeplabv3 import DeepLabHead
from torchvision.models.segmentation.fcn import FCNHead
from torchvision.models.segmentation import deeplabv3_resnet101

# # Pretrained Model
net = deeplabv3_resnet101(pretrained=True, progress=False)
net.classifier = DeepLabHead(2048, 6)
# net.aux_classifier = nn.Sequential()
net.aux_classifier = FCNHead(1024, 6)

# # Non-pretrained Model
# net = deeplabv3_resnet101(pretrained=False, num_classes=6)

trainable_backbone_layers = ['layer4']
for n, p in net.named_parameters():
    if n.startswith('backbone') and n.split('.')[1] not in trainable_backbone_layers:
        p.requires_grad = False

net.to(device)
if torch.cuda.device_count() > 1:
    net = torch.nn.DataParallel(net)
    net.to(device)

torchinfo.summary(net, (1, 3, 256, 256))

Downloading: "https://download.pytorch.org/models/deeplabv3_resnet101_coco-586e9e4e.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet101_coco-586e9e4e.pth
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Layer (type:depth-idx)                             Output Shape              Param #
DeepLabV3                                          --                        --
├─DeepLabHead: 1                                   --                        --
│    └─ASPP: 2                                     --                        --
│    │    └─ModuleList: 3-1                        --                        15,206,912
├─IntermediateLayerGetter: 1-1                     [1, 2048, 32, 32]         --
│    └─Conv2d: 2-1                                 [1, 64, 128, 128]         9,408
│    └─BatchNorm2d: 2-2                            [1, 64, 128, 128]         128
│    └─ReLU: 2-3                                   [1, 64, 128, 128]         --
│    └─MaxPool2d: 2-4                              [1, 64, 64, 64]           --
│    └─Sequential: 2-5                             [1, 256, 64, 64]          --
│    │    └─Bottleneck: 3-2                        [1, 256, 64, 64]          75,008
│    │    └─Bottlen

## Loss Network

* More Multi-Label Segmentation Losses: https://jeune-research.tistory.com/entry/Loss-Functions-for-Image-Segmentation-Region-Based-Losses

* See also: https://smp.readthedocs.io/en/latest/losses.html



In [ ]:
from models.loss import BCEDiceIoUWithLogitsLoss2d, BCEDiceIoULoss2d

# Training

## Define Trainer Function

In [ ]:
from utils import train_one_epoch
from lr_scheduler import CosineAnnealingWarmUpRestarts

## Train and Evaluate

In [ ]:
# Lazy-eval iterable dataset: do not set sampler or shuffle options
num_epoch = 100

batch_size = 35
num_workers = 1

train_loader = DataLoader(train_dataset, batch_size, num_workers=num_workers, drop_last=False)
val_loader = DataLoader(val_dataset, batch_size, num_workers=num_workers, drop_last=False)


loss_func = BCEDiceIoUWithLogitsLoss2d()
optimizer = optim.Adam(net.parameters(), lr=1e-6)
scheduler_config = {
    'T_0': 10, 'T_mult': 2, 'eta_max': 1e-3, 'T_up': 3, 'gamma': 0.5
}
lr_scheduler = CosineAnnealingWarmUpRestarts(optimizer, **scheduler_config)


def load_state_dict(d):
    net.load_state_dict(d['model'])
    optimizer.load_state_dict(d['optimizer'])
    lr_scheduler.load_state_dict(d['lr_scheduler'])


def state_dict():
    from collections import OrderedDict
    d = OrderedDict()
    d['model'] = net.state_dict()
    d['optimizer'] = optimizer.state_dict()
    d['lr_scheduler'] = lr_scheduler.state_dict()
    return d


In [ ]:
import uuid
checkpoint_dir = f'checkpoint/{net.__class__.__name__}-{uuid.uuid4()}'
os.makedirs('checkpoint', exist_ok=True)
for ep in range(num_epoch):
    train_one_epoch(net, loss_func, optimizer, lr_scheduler, train_loader, val_loader, device, ep, warmup_start=False)
    torch.save(state_dict(), os.path.join(checkpoint_dir, 'epoch{}.pt').format(ep))

## Test

In [ ]:
net.eval()

DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se